In [52]:

#import pickle
import pandas as pd
import requests
#import re
#import time

In [53]:
read_transparency = False
get_urls = False
reduce_size = False
provider_table = True
rates_table = True

Content table

In [54]:
import requests
import json

url = 'https://d25kgz5rikkq4n.cloudfront.net/cost_transparency/mrf/table-of-contents/reporting_month=2023-07/2023-07-01_cigna-health-life-insurance-company_index.json?Expires=1692061549&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9kMjVrZ3o1cmlra3E0bi5jbG91ZGZyb250Lm5ldC9jb3N0X3RyYW5zcGFyZW5jeS9tcmYvdGFibGUtb2YtY29udGVudHMvcmVwb3J0aW5nX21vbnRoPTIwMjMtMDcvMjAyMy0wNy0wMV9jaWduYS1oZWFsdGgtbGlmZS1pbnN1cmFuY2UtY29tcGFueV9pbmRleC5qc29uIiwiQ29uZGl0aW9uIjp7IkRhdGVMZXNzVGhhbiI6eyJBV1M6RXBvY2hUaW1lIjoxNjkyMDYxNTQ5fX19XX0_&Signature=QMVzvxRjCyPRwvlg7tHoHvIvIj6msY0ByYpq1YVE4X1oPvaBZ5WyrikOGnzH0XftspdxBG4VXQWPRfWcwJVCzhHty720MbkCaSruRzQz6Au7QADGaAcgj6rXbtknVHDtO1AwbcFZB5SvXPAdtmB8ZoKaougVK4r6bV-su3UetGma50IEk-dFJccY3m10nOFT1syUBF21v2TLJ9rDf3hvRDbg0k5C04IoL7LbcR-v2lHjFqlWCi~~EeBg8SRkK2D0CH45-P1fB2b8edWydpIfPQ8tmAZPECv8Sj5w6E1HFDpUIAtT6~csRN-QNtItFmcWtVvF7UOjUdUUrERgl4YLJA__&Key-Pair-Id=K1NVBEPVH9LWJP'

# Load the json file into a pandas DataFrame
df = pd.read_json(url)


In [55]:
# Normalize the 'reporting_structure' column into a separate DataFrame for 'reporting_plans'
df_structure = pd.json_normalize(df['reporting_structure'], record_path=['reporting_plans'],
                                 meta=[['allowed_amount_file', 'description'],
                                       ['allowed_amount_file', 'location'],
                                       'in_network_files'],
                                 errors='ignore',
                                 record_prefix='plan_')

# Concatenate the normalized structure DataFrame with the original DataFrame (excluding the 'reporting_structure' column)
df = pd.concat([df.drop(['reporting_structure'], axis=1), df_structure], axis=1)

# Now let's handle the 'in_network_files' field separately
# First, let's explode the 'in_network_files' list to get a row for each dictionary it contains
df = df.explode('in_network_files')

# Flatten the 'in_network_files' dictionaries into separate columns
df_in_network_files = pd.json_normalize(df['in_network_files'])

# Reset the index of the DataFrame to avoid duplicate index values
df.reset_index(drop=True, inplace=True)

# Merge the 'in_network_files' DataFrame with the original DataFrame
df = pd.concat([df.drop(['in_network_files'], axis=1), df_in_network_files], axis=1)

df.head()


,reporting_entity_name,reporting_entity_type,plan_plan_name,plan_plan_id_type,plan_plan_id,plan_plan_market_type,allowed_amount_file.description,allowed_amount_file.location,description,location
0,Cigna Health Life Insurance Company,Health Insurance Issuer,PATHWELL OAP County of Rockbridge,ein,546001578,group,allowed amount file,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...,NaN,NaN
1,Cigna Health Life Insurance Company,Health Insurance Issuer,"NATIONAL OAP SAFARILAND, LLC",ein,592044869,group,allowed amount file,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...,NaN,NaN
2,Cigna Health Life Insurance Company,Health Insurance Issuer,PATHWELL OAP Bemas Construction,ein,200027225,group,allowed amount file,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...,NaN,NaN
3,Cigna Health Life Insurance Company,Health Insurance Issuer,LOCALPLUS Bemas Construction,ein,200027225,group,allowed amount file,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...,NaN,NaN
4,Cigna Health Life Insurance Company,Health Insurance Issuer,NATIONAL OAP CITATION OIL & GAS CORP.,ein,742163444,group,allowed amount file,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...,NaN,NaN


In [56]:
df.columns = df.columns.str.replace('.','_')
df.columns

Index(['reporting_entity_name', 'reporting_entity_type', 'plan_plan_name',
       'plan_plan_id_type', 'plan_plan_id', 'plan_plan_market_type',
       'allowed_amount_file_description', 'allowed_amount_file_location',
       'description', 'location'],
      dtype='object')

In [57]:
plan_map = df[df.description == 'in-network file'].copy()
plan_map = dict(zip(plan_map.plan_plan_name,plan_map.location))

urls = df.loc[:,['location']].drop_duplicates()
urls = urls.reset_index(drop=True)
urls

,location
0,NaN
1,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...
2,https://eo13813reform.sagamorehn.com/public/fo...
3,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...
4,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...
...,...
96,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...
97,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...
98,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...
99,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...


In [58]:
urls['sizeMb'] = 0
urls['date'] = 0
urls

,location,sizeMb,date
0,NaN,0,0
1,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...,0,0
2,https://eo13813reform.sagamorehn.com/public/fo...,0,0
3,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...,0,0
4,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...,0,0
...,...,...,...
96,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...,0,0
97,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...,0,0
98,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...,0,0
99,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...,0,0


In [64]:
urls.loc[1,'location']

'https://d25kgz5rikkq4n.cloudfront.net/cost_transparency/mrf/in-network-rates/reporting_month=2023-07/2023-07-01_cigna-health-life-insurance-company_national-oap_in-network-rates.json.gz?Expires=2008700460&Signature=jyQVPdkOzH5ZBe9MLf8XViMH68PNMwoMfmgtI7~8rM3kTWnsY5n-jf9avDjflVlZl~fy7qpvD9pEJCW5vyf1rhQn31qodfYmD5kWkEBN3eNHXmd158Sf7uWzQQFloGSp9lQaZUuWvdGG9Tm5x8d86ZPYJN3eHGDRdxgAi4reH78UmlnNERv4By9E7Fd~NhRt0tbMXs5BnOzPdM6lFAVPvOrLyhrO-D5KL485NsmgAtlHJ3v1OG9wR-BBTaXYR5E1pR4Qt17hV3ZJiX3hMR5kdOfOOz61S29LOcRktY9rXKuHwPmdeV7zkJB2PFUUeqVeYLUlV8Y57wNX9YrsTe1olA__&Key-Pair-Id=K1NVBEPVH9LWJP'

In [67]:


def get_file_size(url):
    response = requests.head(url)
    file_size = int(response.headers.get('Content-Length', -1))
    file_date = int(response.headers.get('Last-Modified'))
    return file_size, file_date  


for indx in urls.index :
    try:
        url = urls.loc[indx,'location']
        size, f_date = get_file_size(url)
        urls.loc[indx,'sizeMb'] = size  / 1024 / 1024
        urls.loc[indx,'date'] = f_date
    except:
        pass
        
    
urls

,location,sizeMb,date
0,NaN,0.000000e+00,0
1,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...,1.613776e+03,1692166962
2,https://eo13813reform.sagamorehn.com/public/fo...,-9.536743e-07,-1
3,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...,6.988941e+01,73284355
4,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...,6.846205e+01,71787661
...,...,...,...
96,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...,3.782583e+01,39663254
97,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...,4.383707e-01,459665
98,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...,9.693804e-01,1016469
99,https://d25kgz5rikkq4n.cloudfront.net/cost_tra...,5.047922e-01,529313


In [66]:
size

NameError: name 'size' is not defined

In [76]:
response = requests.head( urls.iloc[1,0] )

In [79]:
#int(response.headers.get('Content-Length', -1))

print('Content-Length:', response.headers['Content-Length'])
print('Last-Modified:', response.headers['Last-Modified'])



Content-Length: 1692166962
Content-Type: binary/octet-stream
Last-Modified: Tue, 20 Jun 2023 02:31:30 GMT


{'Content-Type': 'binary/octet-stream', 'Content-Length': '1692166962', 'Connection': 'keep-alive', 'Date': 'Tue, 25 Jul 2023 19:44:04 GMT', 'Last-Modified': 'Tue, 20 Jun 2023 02:31:30 GMT', 'ETag': '"0e8fbf070571469fddfae75c9c735b5b-323"', 'x-amz-server-side-encryption': 'AES256', 'x-amz-version-id': 'Ab1ywQM.LTGI8MH9GgfFyt96GFUGJyxs', 'Accept-Ranges': 'bytes', 'Server': 'AmazonS3', 'X-Cache': 'Miss from cloudfront', 'Via': '1.1 4b800f7fa2c3fbb9f4f3c505b0df315e.cloudfront.net (CloudFront)', 'X-Amz-Cf-Pop': 'HIO52-P1', 'X-Amz-Cf-Id': 'AXyuyAo6UhnBVe-zovqSlytxHZNjM0HrHL9Oljl5IrhMwyg3t_aeLg=='}